## Initial steps and preprocessing


In [12]:
import os

datasets_path = os.path.join('/home/vagrant', 'datasets')

In [13]:
file_path= os.path.join(datasets_path, 'ml-latest-small', 'Final_input.csv')

Shopback_Raw_data = sc.textFile(file_path)
Shopback_Raw_data_header = Shopback_Raw_data.take(1)[0]

In [14]:
Shopback_data = Shopback_Raw_data.filter(lambda line: line!=Shopback_Raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[4])).cache()

In [18]:
Shopback_data.take(10)

[(u'1', u'16', u'0'),
 (u'1', u'1', u'0'),
 (u'1', u'4', u'0'),
 (u'1', u'5', u'0'),
 (u'1', u'6', u'0'),
 (u'1', u'2', u'0'),
 (u'1', u'3', u'0'),
 (u'2', u'1', u'0'),
 (u'4', u'6', u'0'),
 (u'10', u'2', u'0.25')]

In [17]:
Store_file_path = os.path.join(datasets_path, 'ml-latest-small', 'Merchant_name.csv')

Store_raw_data = sc.textFile(Store_file_path)
Store_raw_data_header = Store_raw_data.take(1)[0]

Store_raw_data = Store_raw_data.filter(lambda line: line!=Store_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
Store_raw_data.take(3)

[(u'1', u'Groupon'), (u'2', u'FOODPANDA'), (u'3', u'Agoda')]

## Splitting data into training , testing and validating 

In [19]:
training_RDD, validation_RDD, test_RDD = Shopback_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [20]:
training_RDD.take(3)

[(u'1', u'1', u'0'), (u'1', u'4', u'0'), (u'1', u'2', u'0')]

In [21]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.213921321339
For rank 8 the RMSE is 0.21356849342
For rank 12 the RMSE is 0.213902133808
The best model was trained with rank 8


In [22]:
predictions.take(3)

[((29270, 4), 0.08807054802294778),
 ((63852, 4), 0.0),
 ((101022, 4), 0.28659708580905474)]

In [23]:
rates_and_preds.take(3)

[((20273, 9), (0.0, 0.0)),
 ((144685, 1), (0.0, 0.0)),
 ((155784, 10), (0.0, 0.0))]

In [24]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 0.212789012857


The training and testing RMSE are close. That's model is best model

In [26]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

Users_with_Rating_RDD = (Shopback_data.map(lambda x: (x[1], x[2])).groupByKey())
Users_With_avg_rating_RDD = Users_with_Rating_RDD.map(get_counts_and_averages)
Users_rating_counts_RDD = Users_With_avg_rating_RDD.map(lambda x: (x[0], x[1][0]))

In [31]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,1,0.8), 
     (0,12,0.2), 
     (0,13,0.1), 
     (0,25,0.15),
     (0,16,0.2), 
     (0,17,0.3),
     (0,2,0.5), 
     (0,4,0.4), 
     (0,5,0.7),
     (0,6,0.5)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print 'New user ratings: %s' % new_user_ratings_RDD.take(10)

New user ratings: [(0, 1, 0.8), (0, 12, 0.2), (0, 13, 0.1), (0, 25, 0.15), (0, 16, 0.2), (0, 17, 0.3), (0, 2, 0.5), (0, 4, 0.4), (0, 5, 0.7), (0, 6, 0.5)]


In [32]:
complete_data_with_new_ratings_RDD = Shopback_data.union(new_user_ratings_RDD)

In [33]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print "New model trained in %s seconds" % round(tt,3)

New model trained in 14.513 seconds


In [34]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_merchant_RDD = (Shopback_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_merchant_RDD)

In [35]:
# Transform new_user_recommendations_RDD into pairs of the form 
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_merchant_title_affinity = \
    new_user_recommendations_rating_RDD.join(Store_raw_data).join(Users_rating_counts_RDD)
new_user_recommendations_merchant_title_affinity.take(3)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 1165.0 failed 1 times, most recent failure: Lost task 5.0 in stage 1165.0 (TID 399, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/serializers.py", line 236, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-26-5996d051f76d>", line 3, in get_counts_and_averages
TypeError: unsupported operand type(s) for +: 'int' and 'unicode'

	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:135)


In [36]:
new_user_recommendations_rating_RDD.take(3)

[(18, 0.08161844707769139),
 (18, 0.08161844707769139),
 (18, 0.08161844707769139)]

In [37]:
new_user_recommendations_merchant_title_affinity

PythonRDD[1220] at RDD at PythonRDD.scala:43

In [39]:
Store_raw_data.collect()

[(u'1', u'Groupon'),
 (u'2', u'FOODPANDA'),
 (u'3', u'Agoda'),
 (u'4', u'Courts'),
 (u'5', u'Lazada'),
 (u'6', u'Shopbop'),
 (u'7', u'RedMart'),
 (u'8', u'Rakuten'),
 (u'9', u'AliExpress'),
 (u'10', u'ASOS'),
 (u'11', u'Apple'),
 (u'12', u'Luxola'),
 (u'13', u'Expedia'),
 (u'14', u'HipVan'),
 (u'15', u'ZALORA'),
 (u'16', u'StrawberryNET'),
 (u'17', u'iHerb'),
 (u'18', u'Melissa Shoes'),
 (u'19', u'Deal.com.sg'),
 (u'20', u'Taobao'),
 (u'21', u'HotelClub.com'),
 (u'22', u'ebay'),
 (u'23', u'ZUJI'),
 (u'24', u'Booking.com'),
 (u'25', u'Hotels.com'),
 (u'26', u'Muji')]